<a href="https://colab.research.google.com/github/14Emanuel/yaya-ai-engine/blob/main/yaya_job_matching_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Yaya Job Matching Engine (CrewAI + Ollama MVP)

This Colab notebook is a prototype for **IAIA's smart job-matching engine** designed to intelligently match **construction jobs** with the best-fit **local workers** based on:

- ✅ Required skills
- ✅ Tool proficiency
- ✅ Relevant experience
- ✅ Past reviews

---

### 🧠 What Powers This Matching?

- **[CrewAI](https://github.com/joaomdmoura/crewai)** – Multi-agent orchestration framework
- **[Ollama](https://ollama.com)** – Local runtime for open-source language models like `llama3`, `mistral`, etc.
- **LangChain** – Used under the hood to bridge Ollama models with CrewAI agents

---

### 🔧 How It Works

This notebook runs a **simulated hiring flow** with a team of AI agents:

| Agent Role              | Function                                                                 |
|-------------------------|--------------------------------------------------------------------------|
| 🧾 **Job Analyzer**       | Extracts required skills, tools, location from the job post              |
| 🛠️ **Skill Matcher**      | Compares worker skills and tools against job requirements                |
| 🧪 **Experience Validator** | Evaluates relevance and recency of worker history                        |
| ⭐ **Review Evaluator**    | Analyzes reviews to rate worker reliability                             |
| 🧠 **Final Matcher**       | Aggregates all scores and ranks top 3 workers                           |

---

### 📦 Input Data

- 1 sample job post
- 2 worker profiles (can scale up later)

---

### 🚀 Output

After running all cells, you'll get:

- Agent-by-agent reasoning
- Final top 3 worker matches with scores
- Printout of how each worker ranked and why

---

### 🛠️ Requirements

- `crewai` (agent framework)
- `ollama` (Python client for LLMs)
- Local Ollama server running (e.g. `ollama run llama3`) *(outside of Colab)*

---

💡 You can adapt this notebook to real data, add UI, or connect to APIs once the logic is validated.



🧪 CELL 1 – Install Required Packages

In [ ]:
# 📦 Install CrewAI and Ollama for local model usage
!pip install -q crewai ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.1/327.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.2 MB/s e

📦 CELL 2 – Sample Job & Worker Data

In [ ]:
# 🧱 Sample job post and sample worker profiles for testing

job_post = {
    "title": "Tiler Needed for 3 Days in Kisumu",
    "skills": ["tiling", "grouting"],
    "location": "Kisumu",
    "duration": "3 days",
    "description": "Need a tiler to fix bathroom tiles in 3 apartments"
}

worker_profiles = [
    {
        "name": "John Otieno",
        "skills": ["tiling", "plastering"],
        "tools": ["trowel", "tile cutter"],
        "location": "Kisumu",
        "experience": ["tiling 2 bathrooms in March", "kitchen tiling Jan"],
        "reviews": [{"rating": 5, "comment": "Fast and clean job"}]
    },
    {
        "name": "Peter Kamau",
        "skills": ["painting", "welding"],
        "tools": ["spray gun"],
        "location": "Nairobi",
        "experience": ["wall painting", "welding"],
        "reviews": [{"rating": 4, "comment": "Good but slow"}]
    }
]

🧠 CELL 3 – Define the Agents

In [ ]:
# 🧠 CrewAI Agents: Job Analyzer, Skill Matcher, Experience Validator, Review Evaluator, Final Matcher

from crewai import Agent

llm_config = {"model": "llama3"}  # Ollama model name

job_analyzer = Agent(
    role="Job Analyzer",
    goal="Understand job requirements, including key skills and location",
    backstory="You’re an expert at parsing job posts and extracting key data for matching.",
    verbose=True,
    llm_config=llm_config
)

skill_matcher = Agent(
    role="Skill Matcher",
    goal="Match job-required skills with workers' proficiencies",
    backstory="You're skilled at matching technical abilities and tools with job needs.",
    verbose=True,
    llm_config=llm_config
)

experience_validator = Agent(
    role="Experience Validator",
    goal="Evaluate how recent and relevant each worker's experience is to the job",
    backstory="You assess work history and determine best fits based on experience.",
    verbose=True,
    llm_config=llm_config
)

review_evaluator = Agent(
    role="Review Evaluator",
    goal="Score each worker’s reliability and trustworthiness from reviews",
    backstory="You read between the lines in ratings and testimonials to spot dependable workers.",
    verbose=True,
    llm_config=llm_config
)

final_matcher = Agent(
    role="Final Matcher",
    goal="Aggregate all inputs and rank top 3 worker matches",
    backstory="You make final hiring decisions based on everyone’s evaluations.",
    verbose=True,
    llm_config=llm_config
)

📋 CELL 4 – Define the Tasks

In [ ]:
# 🧩 Define CrewAI Tasks for each Agent based on job + worker data

from crewai import Task

def build_tasks(job, workers, agents):
    return [
        Task(
            description=f"Analyze the following job and extract required skills, tools, and location:\n\n{job}",
            expected_output="List of required skills, location, and job duration.",
            agent=agents['job_analyzer']
        ),
        Task(
            description=f"Compare job needs to the following workers:\n\n{workers}\n\nMatch based on skills/tools.",
            expected_output="Skill match score for each worker (0-10) with reasoning.",
            agent=agents['skill_matcher']
        ),
        Task(
            description=f"Evaluate workers' experience relevance for this job:\n\nJob: {job['description']}\nWorkers: {workers}",
            expected_output="Experience match score (0-10) for each worker with explanation.",
            agent=agents['experience_validator']
        ),
        Task(
            description=f"Assess the following worker reviews for reliability:\n\n{workers}",
            expected_output="Trust score (0-5) for each worker with comments.",
            agent=agents['review_evaluator']
        ),
        Task(
            description="Aggregate all agent insights and return the top 3 matching workers with ranking and reason.",
            expected_output="Top 3 worker names, ranked with final scores and justification.",
            agent=agents['final_matcher']
        )
    ]


🚀 CELL 5 – Run the Crew Matching Engine

In [ ]:
# 🚀 Run the CrewAI Matching Engine with Cleaned Inputs (Final Version)

from crewai import Crew, Process, Task

# ✅ Clean formatting for job post (avoid raw JSON/escape issues)
job_str = f"""
Title: {job_post['title']}
Location: {job_post['location']}
Duration: {job_post['duration']}
Skills Required: {', '.join(job_post['skills'])}
Description: {job_post['description']}
"""

# ✅ Clean formatting for worker profiles
workers_str = ""
for worker in worker_profiles:
    reviews = ' | '.join([f"{r['rating']}⭐ - {r['comment']}" for r in worker['reviews']])
    workers_str += f"""
Name: {worker['name']}
Location: {worker['location']}
Skills: {', '.join(worker['skills'])}
Tools: {', '.join(worker['tools'])}
Experience: {', '.join(worker['experience'])}
Reviews: {reviews}
---
"""

# ✅ Define agents in a dictionary
agents = {
    'job_analyzer': job_analyzer,
    'skill_matcher': skill_matcher,
    'experience_validator': experience_validator,
    'review_evaluator': review_evaluator,
    'final_matcher': final_matcher
}

# ✅ Build CrewAI tasks using clean job and worker strings
def build_tasks_clean(job_text, workers_text, agents):
    return [
        Task(
            description=f"Analyze this job post and extract key skills, tools, and location:\n{job_text}",
            expected_output="List of required skills, tools, location, and job duration.",
            agent=agents['job_analyzer']
        ),
        Task(
            description=f"Compare the job requirements with the following workers and score each based on skill + tool fit:\n{workers_text}",
            expected_output="Skill match score (0–10) for each worker with short explanation.",
            agent=agents['skill_matcher']
        ),
        Task(
            description=f"Rate each worker's past experience for how well it fits this job:\nJob Description: {job_post['description']}\nWorkers:\n{workers_text}",
            expected_output="Experience match score (0–10) for each worker and why.",
            agent=agents['experience_validator']
        ),
        Task(
            description=f"Evaluate worker reviews and give a trust score from 0 to 5:\n{workers_text}",
            expected_output="Trust score (0–5) with brief comments per worker.",
            agent=agents['review_evaluator']
        ),
        Task(
            description="Using all previous evaluations, rank the top 3 best-matching workers. Justify the ranking.",
            expected_output="Top 3 workers, ranked with final scores and reasons.",
            agent=agents['final_matcher']
        )
    ]

# ✅ Build the tasks
tasks = build_tasks_clean(job_str, workers_str, agents)

# ✅ Create the Crew
crew = Crew(
    agents=list(agents.values()),
    tasks=tasks,
    process=Process.sequential,
    verbose=True
)

# ✅ Run the Crew and print the result
result = crew.kickoff()

print("\n🟢 FINAL MATCH RESULT:\n")
print(result)



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 17bda175-4895-4652-ab3d-bf04af66fc2a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Job Analyzer                                                                                            │
│                                                                                                                 │
│  Task: Analyze this job post and extract key skills, tools, and location:                                       │
│                                                                                                                 │
│  Title: Tiler Needed for 3 Days in Kisumu                                                                       │
│  Location: Kisumu                                                                                               │
│  Duration: 3 days                                                                                               │
│  Skills Required: tiling, grouting                                                                              │
│  Description: Need a tiler to fix bathroom tiles in 3 apartments                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new

LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

Output()

...

ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable



 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: ad944fd6-055d-4e2e-acce-a5f42046ed10                                                                     │
│  Agent: Job Analyzer                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 17bda175-4895-4652-ab3d-bf04af66fc2a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable